# Trajectory Generalization & Aggregation

In [ ]:
%matplotlib inline

In [ ]:
import urllib
import os
import numpy as np
import pandas as pd
from geopandas import GeoDataFrame, read_file
from shapely.geometry import Point, LineString, Polygon, MultiPoint
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle

import sys
sys.path.append("..")
import movingpandas as mpd

import warnings
warnings.simplefilter("ignore")

## Ship movements (AIS data)

In [ ]:
df = read_file('data/demodata_ais.gpkg')
df['t'] = pd.to_datetime(df['Timestamp'], format='%d/%m/%Y %H:%M:%S')
df = df.set_index('t')
df = df[df.SOG>0]
df.size

In [ ]:
MIN_LENGTH = 100 # meters
traj_collection = mpd.TrajectoryCollection(df, 'MMSI', min_length=MIN_LENGTH)
print("Finished creating {} trajectories".format(len(traj_collection)))

In [ ]:
trips = traj_collection.split_by_observation_gap(timedelta(minutes=5))
print("Extracted {} individual trips from {} continuous vessel tracks".format(len(trips), len(traj_collection)))

In [ ]:
aggregator = mpd.TrajectoryCollectionAggregator(trips, max_distance=1000, min_distance=100, min_stop_duration=timedelta(minutes=5))

In [ ]:
pts = aggregator.get_significant_points_gdf()
clusters = aggregator.get_clusters_gdf()
( pts.hvplot(geo=True, tiles='OSM', frame_width=800) * 
  clusters.hvplot(geo=True, color='red') )

In [ ]:
flows = aggregator.get_flows_gdf()

In [ ]:
( trips.hvplot(color='gray')  *
  flows.hvplot(geo=True, hover_cols=['weight'], line_width='weight', alpha=0.5, color='#1f77b3') * 
  clusters.hvplot(geo=True, color='red', size='n') )

## Bird migration data

In [ ]:
df = read_file('data/demodata_gulls.gpkg')
df['t'] = pd.to_datetime(df['timestamp'])
df = df.set_index('t')
df.size

In [ ]:
traj_collection = mpd.TrajectoryCollection(df, 'individual-local-identifier', min_length=MIN_LENGTH)     
print("Finished creating {} trajectories".format(len(traj_collection)))

In [ ]:
trips = traj_collection.split_by_date(mode='month')
print("Extracted {} individual trips from {} continuous tracks".format(len(trips), len(traj_collection)))

In [ ]:
aggregator = mpd.TrajectoryCollectionAggregator(trips, max_distance=1000000, min_distance=100000, min_stop_duration=timedelta(minutes=5))

In [ ]:
flows = aggregator.get_flows_gdf()
clusters = aggregator.get_clusters_gdf()

( flows.hvplot(geo=True, hover_cols=['weight'], line_width='weight', alpha=0.5, color='#1f77b3', tiles='OSM') * 
  clusters.hvplot(geo=True, color='red', size='n') )